# Importing the condensed graphs

- the condensed graphs are appeared in the `./save` folder.

In [4]:
import numpy as np
import torch
from torch_geometric.data import Data

# example for traditional methods like: random, herding, and k-center:

################ START OF DUMMY DATA #####################
num_nodes = 87
feature_dim = 30
device="cuda:0"
# Random node features (size: [num_nodes, feature_dim])
nodes = torch.randn(num_nodes, feature_dim, dtype=torch.float)
# Random labels for nodes (size: [num_nodes])
labels = torch.randint(0, 5, (num_nodes,), dtype=torch.long)  # Assuming 5 classes
# Random edge index (size: [2, num_edges])
num_edges = 200
edges = torch.randint(0, num_nodes, (2, num_edges), dtype=torch.long)

condensed_graph_path = "save/random/idx_cornell_0.5_15.npy"
################ END OF DUMMY DATA #####################

# supposed this the full graph produced by the flash
graph = Data(
    x=torch.tensor(nodes, dtype=torch.float).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device),
    edge_index=torch.tensor(edges, dtype=torch.long).to(device),
)
graph.n_id = torch.arange(graph.num_nodes)
mask = torch.tensor([True] * graph.num_nodes, dtype=torch.bool)


# this is the condensed graph based on the choosen indices in the npy file.
selected_indices = np.load(condensed_graph_path)
selected_indices_tensor = torch.tensor(selected_indices, dtype=torch.long).to(device)

subset_mask = torch.zeros(graph.num_nodes, dtype=torch.bool).to(device)
subset_mask[selected_indices_tensor] = True

graph_subset = Data(
    x=graph.x[subset_mask],
    y=graph.y[subset_mask],
    edge_index=graph.edge_index[
        :, subset_mask[graph.edge_index[0]] & subset_mask[graph.edge_index[1]]
    ],
)

graph_subset.n_id = torch.arange(graph.num_nodes)
mask_subset = torch.tensor([True]*graph_subset.num_nodes, dtype=torch.bool)

# Similar to in [20] https://github.com/Aisuko/flash_ids/blob/experiments/Theia.ipynb

# for m_n in range(20):
#     loader = NeighborLoader(
#         graph_subset, num_neighbors=[-1, -1], batch_size=5000, input_nodes=mask_subset #try to lower the batch_size
#     )
#     total_loss = 0
#     for subg in loader:
#         model.train()
#         optimizer.zero_grad()
#         out = model(subg.x, subg.edge_index)
#         loss = criterion(out, subg.y)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item() * subg.batch_size
#     print(total_loss / mask.sum().item())

#     loader = NeighborLoader(
#         graph_subset, num_neighbors=[-1, -1], batch_size=5000, input_nodes=mask_subset #try to lower the batch_size
#     )
#     for subg in loader:
#         model.eval()
#         out = model(subg.x, subg.edge_index)

#         sorted, indices = out.sort(dim=1, descending=True)
#         conf = (sorted[:, 0] - sorted[:, 1]) / sorted[:, 0]
#         conf = (conf - conf.min()) / conf.max()

#         pred = indices[:, 0]
#         cond = (pred == subg.y) | (conf >= 0.9)
#         mask[subg.n_id[cond]] = False

#     torch.save(model.state_dict(), f"lword2vec_gnn_theia{m_n}_E3.pth")
#     print(f"Model# {m_n}. {mask.sum().item()} nodes still misclassified \n")

torch.Size([87, 30])
torch.Size([87])


C:\Users\falih\AppData\Local\Temp\ipykernel_36820\2561472706.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(nodes, dtype=torch.float).to(device),
C:\Users\falih\AppData\Local\Temp\ipykernel_36820\2561472706.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(labels, dtype=torch.long).to(device),
C:\Users\falih\AppData\Local\Temp\ipykernel_36820\2561472706.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(edges, dtype=torch.long).to(device),


# The code below is an example of other graph distillation techniques (eg., SGDD, GCOND, etc.)

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

# example for modern graph distillation techniques (eg., SGDD, GCOND, etc.)

################ START OF DUMMY DATA #####################

adj_path = "save/SGDD/adj_apt_dummy_0.5_15.pt"
adj = torch.load(adj_path)

feature_path = "save/SGDD/feat_apt_dummy_0.5_15.pt"
feats = torch.load(feature_path)

label_path = "save/SGDD/label_apt_dummy_0.5_15.pt"
labels = torch.load(label_path)

print(adj.shape)
print(feats.shape)
print(labels.shape)
device = "cuda:0"

nodes = feats
labels = labels
edges, _ = dense_to_sparse(adj)

################ END OF DUMMY DATA #####################

# Similar to in [20] https://github.com/Aisuko/flash_ids/blob/experiments/Theia.ipynb

graph = Data(
    x=torch.tensor(nodes, dtype=torch.float).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device),
    edge_index=torch.tensor(edges, dtype=torch.long).to(device),
)
graph.n_id = torch.arange(graph.num_nodes)
mask = torch.tensor([True] * graph.num_nodes, dtype=torch.bool)


# for m_n in range(20):

#     loader = NeighborLoader(
#         graph, num_neighbors=[-1, -1], batch_size=5000, input_nodes=mask
#     )
#     total_loss = 0
#     for subg in loader:
#         model.train()
#         optimizer.zero_grad()
#         out = model(subg.x, subg.edge_index)
#         loss = criterion(out, subg.y)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item() * subg.batch_size
#     print(total_loss / mask.sum().item())

#     loader = NeighborLoader(
#         graph, num_neighbors=[-1, -1], batch_size=5000, input_nodes=mask
#     )
#     for subg in loader:
#         model.eval()
#         out = model(subg.x, subg.edge_index)

#         sorted, indices = out.sort(dim=1, descending=True)
#         conf = (sorted[:, 0] - sorted[:, 1]) / sorted[:, 0]
#         conf = (conf - conf.min()) / conf.max()

#         pred = indices[:, 0]
#         cond = (pred == subg.y) | (conf >= 0.9)
#         mask[subg.n_id[cond]] = False

#     torch.save(model.state_dict(), f"lword2vec_gnn_theia{m_n}_E3.pth")
#     print(f"Model# {m_n}. {mask.sum().item()} nodes still misclassified \n")

torch.Size([149, 149])
torch.Size([149, 30])
torch.Size([149])


C:\Users\falih\AppData\Local\Temp\ipykernel_22020\978803369.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(nodes, dtype=torch.float).to(device),
C:\Users\falih\AppData\Local\Temp\ipykernel_22020\978803369.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(labels, dtype=torch.long).to(device),
C:\Users\falih\AppData\Local\Temp\ipykernel_22020\978803369.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(edges, dtype=torch.long).to(device),
